## Optimize the beamline

In this example, we use the `bloptools` package to optimize the beamline.

We set up the simulation as we normally do:

In [ ]:
from sirepo_bluesky import prepare_re_env

%run -i $prepare_re_env.__file__

from sirepo_bluesky.sirepo_bluesky import SirepoBluesky
from sirepo_bluesky.sirepo_ophyd import create_classes

connection = SirepoBluesky("http://localhost:8000")

data, schema = connection.auth("shadow", sim_id="00000002")
classes, objects = create_classes(connection=connection)
globals().update(**objects)

bec.disable_plots()

aperture.horizontalSize.kind = "hinted"
w9.duration.kind = "hinted"

We can then instantiate an `Agent` from the `bloptools` package; because it can run any experiment defined with a Bluesky plan, we can give it any of our Sirepo components to use as degrees of freedom. We define the objectives as to maximize the flux density of the beamline.

In [ ]:
from bloptools.bayesian import Agent, DOF, Objective

dofs = [
    DOF(
        device=kbh.x_rot,
        limits=(-5e-2, +5e-2),
        tags=["kb", "kbv"],
        units="deg",
    ),
    DOF(
        device=kbv.x_rot,
        limits=(-5e-2, +5e-2),
        tags=["kb", "kbv"],
        units="deg",
    ),
]

# composite
objectives = [
    Objective(name="w9_flux", description="total flux", target="max", log=True),
    Objective(name="w9_fwhm_x", description="horizontal FWHM", target="min", log=True),
    Objective(name="w9_fwhm_y", description="vertical FWHM", target="min", log=True),
]


agent = Agent(
    dofs=dofs,
    dets=[w9],
    objectives=objectives,
    db=db,
)

We start by running a scan over a quasi-random sample of input parameters, and plot the result:

In [ ]:
RE(agent.learn(acq_func="qr", n=16))
agent.plot_objectives()

We then use a more intelligent strategy ("$q$-expected mean") to narrow in on the optimum.

In [ ]:
RE(agent.learn("qem", iterations=4))

We can see the convergence of our model:

In [ ]:
agent.plot_history()